In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Activation
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess data
x_train = x_train.astype('float32') / 255.0
# x_train = [preprocess_input(i) for i in x_train]
x_test = x_test.astype('float32') / 255.0
# x_test = [preprocess_input(i) for i in x_test]
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)




In [6]:
# Load VGG16 model without the top layer (set include_top=False)
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers for CIFAR-10
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(10, activation='softmax')(x)
x = Activation('softmax', name='final_softmax')(x)  # Add the softmax layer

# Create the new model
model = Model(inputs=vgg_base.input, outputs=x)

# Unfreeze the base layers for fine-tuning
for layer in vgg_base.layers:
    layer.trainable = True

# Re-compile the model for fine-tuning with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine_tune = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=64, epochs=1, verbose=1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 693s 879ms/step - accuracy: 0.4250 - loss: 2.0412 - val_accuracy: 0.5643 - val_loss: 1.8973
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 121ms/step - accuracy: 0.5625 - loss: 1.8988
Test accuracy: 0.564300000667572


In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ final_softmax (Activation)      │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,947,424 (171.46 MB)

 Trainable params: 14,982,474 (57.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 29,964,950 (114.31 MB)

In [23]:
# neurons = []
# for li, l in enumerate(model.layers):
#     for ni in range(l.output.shape[-1]):
#         neurons.append((li, ni))
# for li, l in enumerate(model.layers):
#     for ni in range(l.output.shape[-1]):
#         print("layer: " + str(l))
#         print("layer index: " + str(li))
#         print("neuron: " + str(ni))
# neurons[6723]

layer: <InputLayer name=input_layer_3, built=True>
layer index: 0
neuron: 0
layer: <InputLayer name=input_layer_3, built=True>
layer index: 0
neuron: 1
layer: <InputLayer name=input_layer_3, built=True>
layer index: 0
neuron: 2
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 0
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 1
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 2
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 3
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 4
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 5
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 6
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 7
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 8
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
neuron: 9
layer: <Conv2D name=block1_conv1, built=True>
layer index: 1
n

(21, 0)

In [8]:
model.save('vgg16EX6.keras')